# **Welcome to MAPSHB-Ligand**

This is a Jupyter notebook for running the MAPSHB model using the R and python languages. This notebook implements the model detailed in the paper "***Chemical Features and Machine Learning Assisted Predictions of Protein-Ligand Short Hydrogen Bonds***" ([link here](https://www.researchsquare.com/article/rs-2895170/v1)).

---
**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/shengminz/MAPSHB_Ligand/issues

**Emails**
- **Shengmin Zhou** ([shengminzhou93@gmail.com](mailto:shengminzhou93@gmail.com))
- **Yuanhao Liu** ([yl1398@scarletmail.rutgers.edu](mailto:yl1398@scarletmail.rutgers.edu))
- **Sijian Wang** ([sijian.wang@stat.rutgers.edu](mailto:sijian.wang@stat.rutgers.edu))
- **Lu Wang** ([lwang@chem.rutgers.edu](mailto:lwang@chem.rutgers.edu))

In [ ]:
#@title **Install Conda Colab**
#@markdown It will automatically restart the kernel (session) after finishing the installation. After completion, you may see the following message in the bottom right of your browser:

#@markdown Your session crashed. Automatically restarting.

#@markdown Your session restarted after a crash. Diagnosing...

#@markdown Your session crashed for an unknown reason. View runtime logs

#@markdown It doesn't matter and don't worry. Just continue to run the next cell.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [ ]:
#@title **Install dependencies**
#@markdown Install R. Download MAPSHB-Ligand source codes from Github.

#@markdown It will take a few minutes (~2 mins), please wait.

# install dependencies
%%capture
!git clone https://github.com/shengminz/MAPSHB_Ligand.git
!mv MAPSHB_Ligand/* .

#load dependencies
import sys
import os
import urllib.request
import numpy as np

!apt-get update -qq
!apt-get install -y r-base
!pip install -q rpy2
#!conda install -c r r-base -y

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Install the 'gbm' package in R
utils = importr('utils')
utils.install_packages('gbm')

In [ ]:
#@title ### **Import input file**
#@markdown Click in the "Run" buttom and click "Choose Files" to upload your PDB file from local.

#@markdown **Before upload your own PDB file**, make sure you have already read the following instructions about input file:

#@markdown The uploaded protein structure must be in the PDB format. The protein structure must contain all atoms, including the hydrogen atoms in amino acid residues and small molecule ligands. The hydrogen atoms are used for the determination of hydrogen bonds. You can add them to the amino acid residues using a software of your choice prior to this analysis. For example, in ChimeraX, you can type "addh" to add hydrogens, and in AmberTools, you can use "reduce" to add hydrogens. Then you can save the structure as a PDB file.

from google.colab import files

# Upload file from your local machine
uploaded = files.upload()

# Process the uploaded file
for filename in uploaded.keys():
    # Rename the file to "input.pdb"
    new_filename = "input.pdb"
    with open(new_filename, "wb") as f:
        f.write(uploaded[filename])
    print(f"Uploaded file: {filename} (copied as {new_filename})")


Saving 2zoi.pdb to 2zoi.pdb
Uploaded file: 2zoi.pdb (copied as input.pdb)


In [ ]:
#@title ### **Import hydrogen bond information**
#@markdown Please select the information below for protein-ligand hydrogen bond:

#@markdown - Residue type of amino acid
res_aa = 'TYR' #@param ['ARG', 'HIS', 'LYS', 'ASP', 'GLU', 'SER', 'THR', 'ASN', 'GLN', 'TYR', 'TRP'] {type:"string"}
#@markdown - Residue number of amino acid
res_num = 42 #@param {type:"raw"}
#@markdown - Ligand name
lig_name = 'HC4'  #@param {type:"string"}
#@markdown - Residue number of ligand
lig_num = 169 #@param {type:"raw"}
#@markdown - Atom name of ligand (in PDB file)
atom_ligand = "O4'" #@param {type:"string"}
#@markdown - Charge of ligand functional group
chrg_ligand = 0 #@param {type:"raw"}
#@markdown - pKa value of ligand ("N/A" if unknown)
pKa = 5.19 #@param {type:"raw"}
#@markdown - pKb value of ligand ("N/A" if unknown)
pKb = 14.0 #@param {type:"raw"}
#@markdown - logP value of ligand ("N/A" if unknown)
logP = 1.92 #@param {type:"raw"}
#@markdown - Functional group of ligand
ligand = 'OPHH' #@param ['NA', 'NAM', 'NC', 'NIM', 'NPHH', 'NQUA', 'NRH', 'NS', 'OA', 'OAMI', 'OATE', 'OET', 'OIC', 'OICH', 'ON', 'OOL', 'OONE', 'OP', 'OPHH', 'OS'] {type:"string"}

#@markdown Please check the table below about the ligand functional groups and their abbreviations used in the MAPSHB-Ligand model. (Ph represents the phenyl group and R represents an arbitrary functional group)

#@markdown | Ligand functional group | Abbreviation |
#@markdown | --- | --- |
#@markdown | Aromatic (N) | NA |
#@markdown | Amide (N) | NAM |
#@markdown | Nitrile (-CN) | NC |
#@markdown | Imine | NIM |
#@markdown | R-Ph-NH | NPHH |
#@markdown | Guanidine | NQUA |
#@markdown | Amine | NRH |
#@markdown | R-S-N- | NS |
#@markdown | Aromatic (O) | OA |
#@markdown | Amide (O) | OAMI |
#@markdown | Carboxylate | OATE |
#@markdown | Ester (-R) | OET |
#@markdown | Carboxyl (=O) | OIC |
#@markdown | Carboxyl (-OH) | OICH |
#@markdown | R-N-O | ON |
#@markdown | Hydroxyl | OOL |
#@markdown | Ketone/Aldehyde | OONE |
#@markdown | Phosphate | OP |
#@markdown | Phenol | OPHH |
#@markdown | Phenol | OS |



In [ ]:
#@title ### **Run Prediciton**
#@markdown Click in the "Run" buttom to run prediction. (~1 min)

#@markdown Make sure there's no error file "error.log" generated by this step, otherwise check your input file.

%%capture
# Initialize empty lists to store the information
atom_num = []
atom_type = []
residue_type = []
chain = []
residue_num = []
x = []
y = []
z = []

# Read the "input.pdb" file line by line
with open('input.pdb', 'r') as file:
    for line in file:
        # Check if the line starts with "ATOM" or "HETATM" and skip lines with residue names "HOH" or "DOD"
        if (line.startswith('ATOM') or line.startswith('HETATM')) and not line[17:20].strip() in ['HOH', 'DOD']:
            # Extract information from columns
            atom_num.append(int(line[6:11].strip()))
            atom_type.append(line[12:16].strip())
            residue_type.append(line[17:20].strip())
            chain.append(line[21].strip())
            residue_num.append(int(line[22:26].strip()))
            x.append(float(line[30:38].strip()))
            y.append(float(line[38:46].strip()))
            z.append(float(line[46:54].strip()))

# Get the sequence information
for index in range(len(atom_num)):
    if residue_num[index] == res_num - 3:
       res_aa_n3 = residue_type[index]
    if residue_num[index] == res_num - 2:
       res_aa_n2 = residue_type[index]
    if residue_num[index] == res_num - 1:
       res_aa_n1 = residue_type[index]
    if residue_num[index] == res_num + 1:
       res_aa_1 = residue_type[index]
    if residue_num[index] == res_num + 2:
       res_aa_2 = residue_type[index]
    if residue_num[index] == res_num + 3:
       res_aa_3 = residue_type[index]

atom = []
atom_aa = []
chrg_aa = []
for index in range(len(atom_num)):
  if residue_num[index] == lig_num and residue_type[index] == lig_name and atom_type[index] == atom_ligand:
    coord = [x[index], y[index], z[index]]
    coord_lig = np.array(coord)
  if residue_num[index] == res_num:
    if res_aa == 'TYR' and atom_type[index] == 'OH':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(0)
    if res_aa == 'TRP' and atom_type[index] == 'NE1':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'THR' and atom_type[index] == 'OG1':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(0)
    if res_aa == 'SER' and atom_type[index] == 'OG':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(0)
    if res_aa == 'LYS' and atom_type[index] == 'NZ':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(1)
    if res_aa == 'GLU' and atom_type[index] == 'OE1':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(-1)
    if res_aa == 'GLU' and atom_type[index] == 'OE2':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(-1)
    if res_aa == 'ASP' and atom_type[index] == 'OD1':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(-1)
    if res_aa == 'ASP' and atom_type[index] == 'OD2':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(-1)
    if res_aa == 'ARG' and atom_type[index] == 'NE':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(1)
    if res_aa == 'ARG' and atom_type[index] == 'NH1':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(1)
    if res_aa == 'ARG' and atom_type[index] == 'NH2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(1)
    if res_aa == 'GLN' and atom_type[index] == 'OE1':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(0)
    if res_aa == 'GLN' and atom_type[index] == 'NE2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'ALN' and atom_type[index] == 'OD1':
      atom.append(index)
      atom_aa.append('O')
      chrg_aa.append(0)
    if res_aa == 'ALN' and atom_type[index] == 'ND2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HIS' and atom_type[index] == 'ND1':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HIS' and atom_type[index] == 'NE2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HID' and atom_type[index] == 'ND1':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HID' and atom_type[index] == 'NE2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HIE' and atom_type[index] == 'ND1':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HIE' and atom_type[index] == 'NE2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(0)
    if res_aa == 'HIP' and atom_type[index] == 'ND1':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(1)
    if res_aa == 'HIP' and atom_type[index] == 'NE2':
      atom.append(index)
      atom_aa.append('N')
      chrg_aa.append(1)

distances = []
count = 0
dist_min = 999.9
n_min = -1
with open('temp', 'w') as output:
    for index in range(len(atom)):
        coord_aa = [x[atom[index]], y[atom[index]], z[atom[index]]]
        coord_aa = np.array(coord_aa)
        distance = np.linalg.norm(coord_lig - coord_aa)
        distances.append(distance)
        if distance <= 3.2:
            output.write("{:3} {:1} {:4} {:2d} {:2d} {:3} {:3} {:3} {:3} {:3} {:3} {:6.2f} {:6.2f} {:6.2f} {:1} {:4d} {:3} {:4} {:4d} {:6.4f}\n".format(res_aa, atom_aa[index], ligand, chrg_aa[index], chrg_ligand, res_aa_n3, res_aa_n2, res_aa_n1, res_aa_1, res_aa_2, res_aa_3, pKa, pKb, logP, atom_aa[index], res_num, lig_name, atom_ligand, lig_num, distance))
            count += 1
        if distance <= dist_min:
            dist_min = distance
            n_min = index
    if count == 0:
      output.write("{:3} {:1} {:4} {:2d} {:2d} {:3} {:3} {:3} {:3} {:3} {:3} {:6.2f} {:6.2f} {:6.2f} {:1} {:4d} {:3} {:4} {:4d} {:6.4f}\n".format(res_aa, atom_aa[n_min], ligand, chrg_aa[n_min], chrg_ligand, res_aa_n3, res_aa_n2, res_aa_n1, res_aa_1, res_aa_2, res_aa_3, pKa, pKb, logP, atom_aa[n_min], res_num, lig_name, atom_ligand, lig_num, distance))

# Now you can run your R script
!wget https://zenodo.org/record/8248034/files/Copyright.log
!Rscript MAPSHB_ligand.r
#robjects.r.source('boosting_online.r')

In [ ]:
#@title ### **Display and download result**
#@markdown Click in the "Run" buttom to display and download the prediction.

#@markdown An output file "predicit_results.csv" will be downloaded to your computer.

#@markdown The output file is in the comma-separated value (.csv) format. Each line contains the information of a hydrogen bond.

#@markdown The first column includes the index number. The second column includes the residue names, residue numbers and atom names (after the @ sign) of amino acids. The third column includes the names, numbers and atom names of ligands. The fourth column includes the hydrogen bond distance from your input PDB structure. The hydrogen bond distances (R) are for your reference, and they are not used in our predictions.

#@markdown The next column reports the probability of this hydrogen bond to form a SHB. A recommended classification threshold is 0.870. The models identify the hydrogen bond as a SHB if its probability is ≥ the classificatioin threshold, or a NHB if the probability is below the threshold. You can choose a different threshold to define the hydrogen bond class, as discussed on https://wanggroup.rutgers.edu/mapshb-model/the-mapshb-model/17-research/mapshb-model/41-probability-thresholds.
!cat predict_results.csv
files.download(f"predict_results.csv")

"","Amino Acid","Ligand","R from structure (A)","Predicted Probability"
"1","TYR_42@O","HC4_169@O4'",2.5231,0.999726705733277


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>